In [4]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import random

In [81]:
def extract_dropdown_values(url):
    # Fetch the webpage
    response = requests.get(url)

    # Check for successful request
    if response.status_code != 200:
        print("Failed to fetch the webpage: Status code", response.status_code)
        return []
        
    # Initialize BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    #print(soup)
    
    # Find the dropdown menu by the <select> tag or its name/id
    # Adjust the selector as needed based on the specific structure of your webpage
    dropdown = soup.find('select', id='ctl00_ContentPlaceHolder1_Result1_ddlState')  # Example: id might be different
    #print(dropdown)
    # Initialize a list to store constituencies
    constituencies = []
    
    # Iterate over all option tags within the select tag
    if dropdown:
        for option in dropdown.find_all('option'):
            # Get the constituency name and the value attribute
            name = option.text.strip()
            value = option.get('value', '')
            
            # Append the tuple (name, value) to the list
            constituencies.append((name, value))
    
    return constituencies

In [124]:
# Election commission results URL
url = "https://results.eci.gov.in/PcResultGenJune2024/index.htm"

#Get all states
states = extract_dropdown_values(url)
#print(constituencies)

#base url for individual state page
base_state_url = "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-"

#base table url for constituency
baseurl="https://results.eci.gov.in/PcResultGenJune2024/Constituencywise"

#sleep times array to not hit the website continously
sleep_times=[0.1,1,0.5,2,1.5]

#final dataframe to store all tabular data combined
all_df = pd.DataFrame()

state_index = 1
for state_name, state_value in states:
    if state_value:
        #print(f"State Name: {state_name}, Value: {state_value}")
        print(f"State Name: {state_index} - {state_name}")

        #Create the complete url for individual state
        state_url = base_state_url+state_value+'.htm'
        #print(f'State URL: {state_url}')

        #Get all constituencies for the state from the dropdown list
        try:
            constituencies = extract_dropdown_values(state_url)
        except:
            print(f'Could not get dropdown for state: {state_name}, {state_url}')

        #dataframe to collect all state constituencies tables
        state_df = pd.DataFrame()


        constituency_index = 1
        #Get table data for each constituency
        for name, value in constituencies:
            if value:
                table_url = baseurl+value+'.htm'
                print(f'Constituency Name: {constituency_index} - {name}')
                #print(f'Constituency URL: {table_url}')
                try:
                    table_response = pd.read_html(table_url)
                    tmp_df = table_response[0]
                    tmp_df['Constituency']="-".join(name.split('-')[0:-1]).strip()
                    tmp_df['Constituency_Num']=name.split('-')[-1].strip()
                    tmp_df['State'] = state_name
                    tmp_df['table_url']=table_url
                    tmp_df['state_url'] = state_url
                    state_df = pd.concat([state_df,tmp_df])
                except:
                    print(f'Could not get dropdown for Constituency: {name}, {table_url}')
                time.sleep(random.choice(sleep_times))

                constituency_index += 1

        #Save statewise data
        state_df.to_csv(f'2024_{state_name}_ElectionResultsDownload.csv',header=True,index=False)        
        
        #collect all state data into single dataframe
        all_df = pd.concat([all_df,state_df])  
        #all_df = all_df.reset_index()
        all_df = all_df.reset_index(drop=True)
        state_index += 1

all_df.to_csv('2024_All_States_ElectionResultsDownload.csv',header=True,index=False)

State Name: 1 - Andaman & Nicobar Islands
Constituency Name: 1 - Andaman & Nicobar Islands - 1
State Name: 2 - Andhra Pradesh
Constituency Name: 1 - Amalapuram (SC) - 7
Constituency Name: 2 - Anakapalle - 5
Constituency Name: 3 - Ananthapur - 19
Constituency Name: 4 - Araku (ST) - 1
Constituency Name: 5 - Bapatla (SC) - 15
Constituency Name: 6 - Chittoor(SC) - 25
Constituency Name: 7 - Eluru - 10
Constituency Name: 8 - Guntur - 13
Constituency Name: 9 - Hindupur - 20
Constituency Name: 10 - Kadapa - 21
Constituency Name: 11 - Kakinada - 6
Constituency Name: 12 - Kurnoolu - 18
Constituency Name: 13 - Machilipatnam - 11
Constituency Name: 14 - Nandyal - 17
Constituency Name: 15 - Narsapuram - 9
Constituency Name: 16 - Narsaraopet - 14
Constituency Name: 17 - Nellore - 22
Constituency Name: 18 - Ongole - 16
Constituency Name: 19 - Rajahmundry - 8
Constituency Name: 20 - Rajampet - 24
Constituency Name: 21 - Srikakulam - 2
Constituency Name: 22 - Thirupathi(SC) - 23
Constituency Name: 23 -